In [ ]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from itertools import islice

In [ ]:
def cluster_rule_based_prediction(user_id, movie_id, rounded=True):
    cluster = int(movies_hot_df.loc[movie_id]["Cluster"])
    user_pred = users_df.loc[user_id][f"Cluster_mean_{cluster}"]
    class_pred = cluster_ratings[cluster]
    film_pred = movies_hot_df.loc[movie_id]["rating_mean"]
    # return round((user_pred * 0.7 + class_pred * 0.1 + film_pred * 0.3) * 2) / 2
    # return round(film_pred * 2) / 2
    return round(user_pred * 2) / 2 if rounded else user_pred

class ClusterRuleBasedEstimator(BaseEstimator):
    def fit(self, X, y=None):
        return self

    def predict(self, X):
        return np.array([cluster_rule_based_prediction(user, movie) for user, movie in X])


def custom_scorer(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)


def custom_mae_scorer(y_true, y_pred):
    return np.mean(abs(y_true - y_pred))


def custom_accuracy_scorer(y_true, y_pred, tol=(0.5 + 1e-9)):
    accuracy = np.isclose(y_pred, y_true, atol=tol).mean()
    return accuracy

In [ ]:
ratings_df = pd.read_csv("data/ratings.csv")
movies_df = pd.read_csv("data/movies.csv", index_col="movieId")

X = ratings_df.drop(["rating"], axis=1)
y = ratings_df["rating"]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42)

ratings_train_df = pd.concat([X_train, y_train], axis=1)
ratings_test_df = pd.concat([X_test, y_test], axis=1)

n_movie_clusters = 2
n_user_clusters = 100

movies_df['Genres_Split'] = movies_df['genres'].apply(lambda x: x.split('|'))
mlb = MultiLabelBinarizer()
binary_matrix = mlb.fit_transform(movies_df['Genres_Split'])
binary_df = pd.DataFrame(binary_matrix, columns=mlb.classes_)

movies_df.reset_index(inplace=True)
movies_hot_df = pd.concat([movies_df, binary_df], axis=1)
movies_hot_df = movies_hot_df.drop(['genres', 'Genres_Split'], axis=1)
movies_hot_df = movies_hot_df.merge(ratings_train_df.groupby("movieId")["rating"].mean().reset_index(), on="movieId")
movies_hot_df.rename(columns={"rating": "rating_mean"}, inplace=True)
movies_hot_df = movies_hot_df.drop(["title"], axis=1).set_index("movieId")

movies_hot_df["rating_mean"] /= 3

kmeans = KMeans(n_clusters=n_movie_clusters, random_state=42)
movies_hot_df['Cluster'] = kmeans.fit_predict(movies_hot_df)

ratings_with_clusters = ratings_train_df.merge(movies_hot_df, left_on='movieId', right_index=True)

user_cluster_stats = ratings_with_clusters.groupby(['userId', 'Cluster'])['rating'].agg(['count', 'sum']).reset_index()
user_cluster_pivot = user_cluster_stats.pivot(index='userId', columns='Cluster', values=['count', 'sum']).fillna(0)
user_cluster_pivot.columns = [f'Cluster_{stat}_{cluster}' for stat, cluster in user_cluster_pivot.columns]

for cluster in range(n_movie_clusters):
    count_col = f'Cluster_count_{cluster}'
    sum_col = f'Cluster_sum_{cluster}'
    mean_col = f'Cluster_mean_{cluster}'
    if count_col in user_cluster_pivot.columns and sum_col in user_cluster_pivot.columns:
        user_cluster_pivot[mean_col] = user_cluster_pivot[sum_col] / user_cluster_pivot[count_col]
    else:
        user_cluster_pivot[count_col] = 0
        user_cluster_pivot[sum_col] = 0
        user_cluster_pivot[mean_col] = 0

user_cluster_pivot = user_cluster_pivot.fillna(0)
users_df = user_cluster_pivot.reset_index()
users_df.set_index("userId", inplace=True)
users_df.head()
cluster_ratings = ratings_train_df.merge(
    movies_hot_df.reset_index()[["Cluster", "movieId"]],
    on="movieId") \
    .groupby("Cluster")["rating"].mean()

In [ ]:
y_pred, y_true = [], []
unknown_count = 0

for i, row in islice(ratings_test_df.iterrows(), 0, 100_000):
    try:
        y_pred.append(cluster_rule_based_prediction(row["userId"], row["movieId"]))
    except:
        y_pred.append(3.5)
        unknown_count += 1

    y_true.append(row["rating"])

acc_mse = custom_scorer(np.array(y_pred), np.array(y_true))
acc = custom_mae_scorer(np.array(y_pred), np.array(y_true))
print("MSE:", acc_mse)
print("MAE:", acc)